In [2]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize

# Choix des $q_k$ aléatoire

In [3]:
def next_matrix(Q, w0, x0, w, x, max_iter):
    N = len(x)
    Q_tilde = np.zeros((N+1, N+1))
    Q_tilde[1:, 1:] = Q
    Q_tilde[N, 0] = w0
    
    x_tilde = np.zeros(N+1)
    x_tilde[0] = x0
    for i in range(N):
        x_tilde[i+1] = x[i]

    it = 0
    while(it < max_iter):
        is_compatible = True
        q = np.random.randn(N)

        Q_tilde[0, 0] = - np.sum(q)
        for i in range(N):
            Q_tilde[0, i+1] = q[i]

        # vérification de la compatibilité
        Q_tilde_inv = np.linalg.inv(Q_tilde)
        
        # [Q_tilde Diag(x0,x1,...,xN) Q_tilde-1]ij <= 0 pour i != j
        D_tilde = np.diag(x_tilde)
        A = Q_tilde @ D_tilde @ Q_tilde_inv
        for i in range(N+1):
            for j in range(N+1):
                if(i != j and A[i, j] > 0):
                    is_compatible = False
        it += 1
        
        if(is_compatible):
            return Q_tilde
    return None

In [4]:
Q = np.array([[-1, 1],
              [1, 2]])

x = np.array([1, 10])
w = np.array([1, 2])
x0 = 0.5
w0 = 0.3

In [5]:
print(next_matrix(Q, w0, x0, w, x, 10000))

None


# Optimisation pour trouver des $q_k$ convenables

In [6]:
def compute_A(Q, w0, x0, x, q):
    N = len(x)
    
    # Construction de Q_tilde (matrice (N+1)x(N+1))
    Q_tilde = np.zeros((N+1, N+1))
    Q_tilde[1:, 1:] = Q
    Q_tilde[0, 0] = -np.sum(q)
    Q_tilde[0, 1:] = q
    Q_tilde[N, 0] = w0  # note : w n'est pas utilisé ici

    # Construction de x_tilde (vecteur N+1)
    x_tilde = np.zeros(N+1)
    x_tilde[0] = x0
    x_tilde[1:] = x

    try:
        Q_tilde_inv = np.linalg.inv(Q_tilde)
    except np.linalg.LinAlgError:
        raise ValueError("Q_tilde is not invertible.")

    # Calcul de A = Q_tilde @ diag(x_tilde) @ Q_tilde_inv
    D_tilde = np.diag(x_tilde)
    A = Q_tilde @ D_tilde @ Q_tilde_inv

    return A

def phi(q, Q, w0, x0, w, x, penalty=1e6):
    A = compute_A(Q, w0, x0, x, q)
    
    N = A.shape[0]
    total = 0.0
    for i in range(N):
        for j in range(N):
            if i != j:
                if A[i, j] > 0:
                    total += 50 * A[i, j]
                else:
                    total += A[i, j]
    return total

def optimize_q_sum1(Q, w0, x0, w, x, penalty=1e6):
    N = len(x)
    
    def objective(q):
        return phi(q, Q, w0, x0, w, x, penalty)
    
    cons = [
    {'type': 'ineq', 'fun': lambda q: -q[0]},  # q[0] <= 0 ⇒ -q[0] ≥ 0
    {'type': 'ineq', 'fun': lambda q: q[1]}    # q[1] ≥ 0
    ]
    
    q0 = np.random.randn(N)
    q0 = q0 / np.sum(q0)  # normaliser q0 pour respecter la contrainte au départ
    
    res = minimize(
        objective,
        x0=q0,
        method='SLSQP',  # méthode supportant contraintes
        constraints=cons,
        options={'disp': True}
    )
    
    if res.success:
        print("Optimisation réussie")
    else:
        print("Échec de l'optimisation :", res.message)
    
    return res.x, res.fun


q_opt, phi_val = optimize_q_sum1(Q, w0, x0, w, x)
print("q optimal :", q_opt)
print("phi(q_opt) :", phi_val)


Inequality constraints incompatible    (Exit mode 4)
            Current function value: 7.443901746017376e+18
            Iterations: 19
            Function evaluations: 110
            Gradient evaluations: 18
Échec de l'optimisation : Inequality constraints incompatible
q optimal : [ 1.32222619e+18 -1.47128608e+16]
phi(q_opt) : 7.443901746017376e+18


# À la main

In [7]:
def check_neg(Q_tilde, w0, x0, w, x):
    N = len(Q_tilde) - 1
    x_tilde = np.zeros(N + 1)
    x_tilde[0] = x0
    x_tilde[1:] = x
    D = np.diag(x_tilde)
    Q_tilde_inv = np.linalg.inv(Q_tilde)
    M = Q_tilde @ D @ Q_tilde_inv
    print(M)
    print('\n')
    for i in range(N+1):
        for j in range(N+1):
            if(i!=j and M[i,j] > 0):
                return False
    return True


In [8]:
N = 2
x = np.array([1, 10])
w = np.array([1, 2])
x0 = 8
w0 = -3

Q = np.array([[-2, 3],
              [w[0], w[1]]])

q1 = 1
q2 = -4

Q_tilde = np.zeros((N+1, N+1))
Q_tilde[1:, 1:] = Q
Q_tilde[N, 0] = w0
Q_tilde[0,1] = q1
Q_tilde[0,2] = q2
Q_tilde[0,0] = -q1-q2

print(Q_tilde)

print(check_neg(Q_tilde, w0, x0, w, x))

[[ 3.  1. -4.]
 [ 0. -2.  3.]
 [-3.  1.  2.]]
[[-10.5 -15.  -18.5]
 [ 27.   28.   27. ]
 [ -6.5  -3.    1.5]]


False


In [24]:
def test_random_q(Q, w0, x0, x, condition_fn, trials=100):
    """
    Teste 100 valeurs aléatoires de (q1, q2) telles que :
    q1 >= 0, q2 <= 0

    - Q : matrice de base (taille NxN)
    - w0, x0, x : paramètres pour construire Q_tilde
    - condition_fn : fonction prenant Q_tilde et retournant True/False
    - trials : nombre d'essais

    Affiche les cas où la condition est satisfaite.
    """
    N = len(x)
    results = []

    for _ in range(trials):
        q1 = 50*np.random.rand()  
        q2 = -50*np.random.rand()  
        q = np.array([q1, q2])

        # Construction de Q_tilde
        Q_tilde = np.zeros((N + 1, N + 1))
        Q_tilde[1:, 1:] = Q
        Q_tilde[0, 0] = -np.sum(q)
        Q_tilde[0, 1:] = q
        Q_tilde[N, 0] = w0

        try:
            if condition_fn(Q_tilde,  w0, x0, w, x):
                results.append((q1, q2))
        except np.linalg.LinAlgError:
            continue  # Matrice non inversible, on passe

    print(f"Nombre de matrices satisfaisant la condition : {len(results)}")

    return results

N = 2
x = np.array([2, 5])
w = np.array([1, 2])
x0 = 1
w0 = 3

Q = np.array([[2, -2],
              [w[0], w[1]]])

test_random_q(Q, w0, x0, x, check_neg, trials=1000)


[[ 1.24157251 14.59916995  2.90781879]
 [-0.08307688  4.25842749 -1.        ]
 [ 0.12461532 -2.88764123  2.5       ]]


[[  3.08791146  14.0952349  -31.03354507]
 [  0.06727918   2.41208854  -1.        ]
 [ -0.10091877  -0.11813281   2.5       ]]


[[ 0.80845164  2.39636211 -0.16956061]
 [-1.12967486  4.69154836 -1.        ]
 [ 1.69451228 -3.53732253  2.5       ]]


[[  22.65770551 -370.42761365  -20.29734632]
 [   1.06702153  -17.15770551   -1.        ]
 [  -1.60053229   29.23655827    2.5       ]]


[[  3.07388587   7.9641635  -17.24812203]
 [  0.12023836   2.42611413  -1.        ]
 [ -0.18035754  -0.13917119   2.5       ]]


[[ 5.13028732 -6.9648945  -6.24925971]
 [ 0.66092425  0.36971268 -1.        ]
 [-0.99138638  2.94543097  2.5       ]]


[[  3.21627208   8.32299405 -21.82727501]
 [  0.10153682   2.28372792  -1.        ]
 [ -0.15230523   0.07440812   2.5       ]]


[[ 0.07896766 51.21570838 -7.76877367]
 [-0.11855569  5.42103234 -1.        ]
 [ 0.17783354 -4.63154851  2.5       

[]